# Ejercicio 1: Peticiones HTTP usando Sockets

Aquí un ejemplo de petición via sockets

In [4]:
import ssl
import socket

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
s.connect(('google.com', 443))
s = ssl.wrap_socket(s, ssl_version=ssl.PROTOCOL_SSLv23)
s.sendall(b"GET / HTTP/1.1\r\nHost:www.google.com\r\n\r\n")
data = s.recv(1024)
print(data)

b'HTTP/1.1 200 OK\r\nDate: Sat, 24 Apr 2021 16:03:52 GMT\r\nExpires: -1\r\nCache-Control: private, max-age=0\r\nContent-Type: text/html; charset=ISO-8859-1\r\nP3P: CP="This is not a P3P policy! See g.co/p3phelp for more info."\r\nServer: gws\r\nX-XSS-Protection: 0\r\nX-Frame-Options: SAMEORIGIN\r\nSet-Cookie: CONSENT=PENDING+183; expires=Fri, 01-Jan-2038 00:00:00 GMT; path=/; domain=.google.com\r\nAlt-Svc: h3-29=":443"; ma=2592000,h3-T051=":443"; ma=2592000,h3-Q050=":443"; ma=2592000,h3-Q046=":443"; ma=2592000,h3-Q043=":443"; ma=2592000,quic=":443"; ma=2592000; v="46,43"\r\nAccept-Ranges: none\r\nVary: Accept-Encoding\r\nTransfer-Encoding: chunked\r\n\r\n5223\r\n<!doctype html><html itemscope="" itemtype="http://schema.org/WebPage" lang="es"><head><meta content="Google.es permite acceder a la informaci\xf3n mundial en castellano, catal\xe1n, gallego, euskara e ingl\xe9s." name="description"><meta content="noodp" name="robots"><meta content="text/html; charset=UTF-8" http-equiv="Content

Crear un socket que haga una petición a la web [ifconfig.io](http://ifconfig.io) para obtener nuestra IP pública
> Punto extra! formatea la salida para mostrar SOLO LA IP, sin el resto del texto

In [7]:
import socket

with socket.socket() as s:
    s.connect(('ifconfig.io', 80))
    s.sendall(b'GET / HTTP/1.1\r\nHost: ifconfig.io\r\nUser-Agent: curl\r\n\r\n')
    data = s.recv(1024)
    data_str = data.decode('ascii')
    ip = data_str.split('\r\n\r\n')[1].rstrip()
    print(ip)

83.50.235.155


# Ejercicio 2: HTTP protocol client con Python

Establecemos una conexión con la web `www.python.org`

In [26]:
import http.client

conn = http.client.HTTPSConnection("www.python.org")
print(conn)

## Peticiones GET

Lanzamos una petición GET para obtener la información de esa página

In [27]:
conn.request("GET", "/")

A continuación vamos a guardar la respuesta en un objeto e inspeccionar el resultado

In [28]:
r1 = conn.getresponse()
print(r1)

Los campos disponibles en el objeto `HTTPResponse` puedes encontrarlos en la [documentación oficial de Python](https://docs.python.org/3/library/http.client.html#httpresponse-objects).

Vamos a inspeccionar los principales:

In [29]:
print(r1.status, r1.reason)

200 OK


In [30]:
data1 = r1.read()  # This will return entire content.
print(data1)

b'<!doctype html>\n<!--[if lt IE 7]>   <html class="no-js ie6 lt-ie7 lt-ie8 lt-ie9">   <![endif]-->\n<!--[if IE 7]>      <html class="no-js ie7 lt-ie8 lt-ie9">          <![endif]-->\n<!--[if IE 8]>      <html class="no-js ie8 lt-ie9">                 <![endif]-->\n<!--[if gt IE 8]><!--><html class="no-js" lang="en" dir="ltr">  <!--<![endif]-->\n\n<head>\n    <meta charset="utf-8">\n    <meta http-equiv="X-UA-Compatible" content="IE=edge">\n\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jquery/1.8.2/jquery.min.js">\n    <link rel="prefetch" href="//ajax.googleapis.com/ajax/libs/jqueryui/1.12.1/jquery-ui.min.js">\n\n    <meta name="application-name" content="Python.org">\n    <meta name="msapplication-tooltip" content="The official home of the Python Programming Language">\n    <meta name="apple-mobile-web-app-title" content="Python.org">\n    <meta name="apple-mobile-web-app-capable" content="yes">\n    <meta name="apple-mobile-web-app-status-bar-style" content="black"

Ahora vamos a hacer lo mismo con una respuesta inválida, por ejemplo, `docs.python.org/parrot.spam`

In [39]:
conn = http.client.HTTPSConnection("docs.python.org")
conn.request("GET", "/parrot.spam")
r2 = conn.getresponse()
print(r2.status, r2.reason)

404 Not Found


In [42]:
data2 = r2.read()
print(data2)
conn.close()

b''


## Peticiones `POST`

En la web de [http://bugs.python.org/](http://bugs.python.org/) tienen un issue abierto (el [12524](http://bugs.python.org/issue12524)) para que los desarrolladores python puedan hacer pruebas de peticiones POST sobre una web. Vamos a hacer una prueba nosotros.

En este caso tenemos que configurar más parámetros que con el get, ya que no vamos a obtener información sino que vamos a modificar un servicio externo. Por ello, vamos a importar la librería `urllib`, para gestionar mejor la construcción de la URL

In [43]:
import http.client, urllib.parse

A continuación vamos a construir la url y las headers de nuestra petición:

In [45]:
params = urllib.parse.urlencode({'@number': 12524, '@type': 'issue', '@action': 'show'})
headers = {"Content-type": "application/x-www-form-urlencoded",
           "Accept": "text/plain"}

Construimos la conexión y lanzamos la petición:

In [53]:
conn = http.client.HTTPConnection("bugs.python.org")
conn.request("POST", "", params, headers)
response = conn.getresponse()
print(response.status, response.reason)

301 Moved Permanently


Parece que algo no ha ido bien... prueba ahora con una conexión https:

In [54]:
conn = http.client.HTTPSConnection("bugs.python.org")
conn.request("POST", "", params, headers)
response = conn.getresponse()
print(response.status, response.reason)

302 Found


Muestra el contenido de la respuesta:

In [56]:
data = response.read()
data

b''

Y por último, cierra la conexión:

In [49]:
conn.close()

# Ejercicio 3: Librería `requests`

La librería más usada por los desarrolladores Python para hacer requests a una API es **[requests: http para humanos](https://docs.python-requests.org/es/latest/)**
    

In [2]:
! pip install requests

In [3]:
import requests

Uno de los métodos HTTP más comunes es GET. El método GET indica que está intentando obtener o recuperar datos de un recurso específico.

In [4]:
requests.get('https://api.github.com')

<Response [200]>

Una respuesta (response) es un objeto poderoso para inspeccionar los resultados de la petición. Haz la misma petición nuevamente, pero esta vez almacena el valor de retorno en una variable para que puedas ver más de cerca sus atributos y comportamientos:

In [23]:
response = requests.get('https://api.github.com')

In [8]:
response.status_code

200

A veces, puedes usar el campo `status_code` para tomar decisiones en el código

In [4]:
if response.status_code == 200:
    print('Success!')
elif response.status_code == 404:
    print('Not Found.')

Success!


Prueba ahora con una URL inválida

In [5]:
response = requests.get('https://api.github.com/invalid')

In [6]:
response.status_code

404

La respuesta de un GET a menudo tiene información valiosa, conocida como carga útil, en el cuerpo del mensaje. Usando los atributos y métodos de `response`, puede ver la carga útil en una variedad de formatos diferentes.

In [7]:
response = requests.get('https://api.github.com')
response.content

b'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sea

In [8]:
response.text

'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sear

In [10]:
response.json()

{'current_user_url': 'https://api.github.com/user',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}',
 'issues_url': 'https://api.github.com/issues',
 'keys_url': '

## Headers

Los encabezados de respuesta pueden darte información útil, como el tipo de contenido de la carga útil de respuesta y un límite de tiempo sobre cuánto tiempo almacenar en caché la respuesta. Para ver estos encabezados, accede al campo `headers`:

In [11]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Sun, 09 May 2021 00:05:57 GMT', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept, Accept-Encoding, Accept, X-Requested-With', 'ETag': '"27278c3efffccc4a7be1bf315653b901b14f2989b2c2600d7cc2e90a97ffbf60"', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Used, X-RateLimit-Resource, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type, Deprecation, Sunset', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '0', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src 'none'", 'Content-Type': 'application/json; charset=utf-8', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Content-Encoding': 'gzip',

Una forma común de personalizar una solicitud GET es pasar valores a través de parámetros de cadena de consulta en la URL. Para hacer usa la función get utilizando el parámetro `params`. 

Por ejemplo, puede usar la API de búsqueda de GitHub para buscar la biblioteca de solicitudes:

In [2]:
import requests

# Search GitHub's repositories for requests
response = requests.get(
    'https://api.github.com/search/repositories',
    params={'q': 'requests+language:python'},
)

# Inspect some attributes of the `requests` repository
json_response = response.json()
repository = json_response['items'][0]
print(f'Repository name: {repository["name"]}')  # Python 3.6+
print(f'Repository description: {repository["description"]}')  # Python 3.6+

Repository name: grequests
Repository description: Requests + Gevent = <3


## Otros métodos HTTP

In [32]:
response = requests.post('https://httpbin.org/post', data={'key':'value'})
response.json()

{'args': {},
 'data': '',
 'files': {},
 'form': {'key': 'value'},
 'headers': {'Accept': '*/*',
  'Accept-Encoding': 'gzip, deflate',
  'Content-Length': '9',
  'Content-Type': 'application/x-www-form-urlencoded',
  'Host': 'httpbin.org',
  'User-Agent': 'python-requests/2.25.1',
  'X-Amzn-Trace-Id': 'Root=1-607f5990-1c6427264bce135544fba0d0'},
 'json': None,
 'origin': '83.50.235.155',
 'url': 'https://httpbin.org/post'}

In [35]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete')
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

<Response [200]>

## Ejercicio 4: Explora otras librerías

In [39]:
response = requests.get(
    'https://cat-fact.herokuapp.com/facts',
    params={'animal_type': 'horse'},
)
response.json()

[{'status': {'verified': True, 'sentCount': 1},
  'type': 'cat',
  'deleted': False,
  '_id': '58e008800aac31001185ed07',
  'user': '58e007480aac31001185ecef',
  'text': 'Wikipedia has a recording of a cat meowing, because why not?',
  '__v': 0,
  'source': 'user',
  'updatedAt': '2020-08-23T20:20:01.611Z',
  'createdAt': '2018-03-06T21:20:03.505Z',
  'used': False},
 {'status': {'verified': True, 'sentCount': 1},
  'type': 'cat',
  'deleted': False,
  '_id': '58e008630aac31001185ed01',
  'user': '58e007480aac31001185ecef',
  'text': 'When cats grimace, they are usually "taste-scenting." They have an extra organ that, with some breathing control, allows the cats to taste-sense the air.',
  '__v': 0,
  'source': 'user',
  'updatedAt': '2020-08-23T20:20:01.611Z',
  'createdAt': '2018-02-07T21:20:02.903Z',
  'used': False},
 {'status': {'verified': True, 'sentCount': 1},
  'type': 'cat',
  'deleted': False,
  '_id': '58e00a090aac31001185ed16',
  'user': '58e007480aac31001185ecef',
  'text

In [47]:
response = requests.get('https://api.fbi.gov/wanted/v1/list')
data = response.json()
print(data['total'])
print(data['items'][:10])

990
[{'files': [{'url': 'https://www.fbi.gov/wanted/seeking-info/manassas-mall-shooting/download.pdf', 'name': 'English'}], 'age_range': None, 'uid': '902d88d5245d401d8b36e61b646e46df', 'weight': None, 'occupations': None, 'field_offices': ['washingtondc'], 'locations': None, 'reward_text': 'The FBI is offering a reward of up to $5,000 for information leading to the identification, arrest, and conviction of the individual(s) responsible for the murder of Jahmar Latravern Graves.', 'hair': None, 'ncic': None, 'dates_of_birth_used': None, 'caution': None, 'nationality': None, 'age_min': None, 'age_max': None, 'scars_and_marks': None, 'subjects': ['Seeking Information'], 'aliases': None, 'race_raw': None, 'suspects': None, 'publication': '2021-04-20T12:51:00', 'title': 'MANASSAS MALL SHOOTING', 'coordinates': [], 'hair_raw': None, 'languages': None, 'complexion': None, 'build': None, 'details': '<p>The FBI\'s Northern Virginia Safe Streets Violent Crimes Task Force is assisting the Prince

In [55]:
response = requests.get('https://www.fruityvice.com/api/fruit/carbohydrates/', params={"min": "0"})
response.json()

[{'genus': 'Fragaria',
  'name': 'Strawberry',
  'id': 3,
  'family': 'Rosaceae',
  'order': 'Rosales',
  'nutritions': {'carbohydrates': 5.5,
   'protein': 0.8,
   'fat': 0.4,
   'calories': 29,
   'sugar': 5.4}},
 {'genus': 'Musa',
  'name': 'Banana',
  'id': 1,
  'family': 'Musaceae',
  'order': 'Zingiberales',
  'nutritions': {'carbohydrates': 22,
   'protein': 1,
   'fat': 0.2,
   'calories': 96,
   'sugar': 17.2}},
 {'genus': 'Solanum',
  'name': 'Tomato',
  'id': 5,
  'family': 'Solanaceae',
  'order': 'Solanales',
  'nutritions': {'carbohydrates': 3.9,
   'protein': 0.9,
   'fat': 0.2,
   'calories': 74,
   'sugar': 2.6}},
 {'genus': 'Pyrus',
  'name': 'Pear',
  'id': 4,
  'family': 'Rosaceae',
  'order': 'Rosales',
  'nutritions': {'carbohydrates': 15,
   'protein': 0.4,
   'fat': 0.1,
   'calories': 57,
   'sugar': 10}},
 {'genus': 'Prunus',
  'name': 'Cherry',
  'id': 9,
  'family': 'Rosaceae',
  'order': 'None',
  'nutritions': {'carbohydrates': 12,
   'protein': 1,
   'fat

In [3]:
response = requests.get('https://api.imgflip.com/get_memes')
response.json()

{'success': True,
 'data': {'memes': [{'id': '181913649',
    'name': 'Drake Hotline Bling',
    'url': 'https://i.imgflip.com/30b1gx.jpg',
    'width': 1200,
    'height': 1200,
    'box_count': 2},
   {'id': '112126428',
    'name': 'Distracted Boyfriend',
    'url': 'https://i.imgflip.com/1ur9b0.jpg',
    'width': 1200,
    'height': 800,
    'box_count': 3},
   {'id': '87743020',
    'name': 'Two Buttons',
    'url': 'https://i.imgflip.com/1g8my4.jpg',
    'width': 600,
    'height': 908,
    'box_count': 2},
   {'id': '129242436',
    'name': 'Change My Mind',
    'url': 'https://i.imgflip.com/24y43o.jpg',
    'width': 482,
    'height': 361,
    'box_count': 2},
   {'id': '131087935',
    'name': 'Running Away Balloon',
    'url': 'https://i.imgflip.com/261o3j.jpg',
    'width': 761,
    'height': 1024,
    'box_count': 5},
   {'id': '247375501',
    'name': 'Buff Doge vs. Cheems',
    'url': 'https://i.imgflip.com/43a45p.png',
    'width': 937,
    'height': 720,
    'box_count'

Elige una api pública (a poder ser sin token de autorización) y explora sus endpoints con alguna librería de protocolo http

https://github.com/public-apis/public-apis